In [66]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common import exceptions  
from time import sleep
from unidecode import unidecode
from random import randint
import numpy as np

# Do not run this part of the code

In [67]:
drive_path = './Driver/chromedriver.exe'
url = 'https://www.agoda.com/search?area=571269&locale=en-us&ckuid=12f8264b-428c-4a49-a16e-f85616b7f9fe&prid=0&gclid=EAIaIQobChMIjcrjz_2ogAMV1V0rCh13BQDxEAAYASAAEgIPIvD_BwE&currency=PHP&correlationId=2ea4f605-5fb9-4fd0-a3db-b0747e9205d6&analyticsSessionId=3118740912916874695&pageTypeId=103&realLanguageId=1&languageId=1&origin=PH&cid=1891467&tag=1b1738b9-a624-86b7-52bb-979616124dd3&userId=12f8264b-428c-4a49-a16e-f85616b7f9fe&whitelabelid=1&loginLvl=0&storefrontId=3&currencyId=18&currencyCode=PHP&htmlLanguage=en-us&cultureInfoName=en-us&machineName=hk-pc-2g-acm-web-user-7fdb9497b6-8jcl6&trafficGroupId=5&sessionId=nzunic5hsy4ds3nsronvws2d&trafficSubGroupId=122&aid=82361&useFullPageLogin=true&cttp=4&isRealUser=true&mode=production&browserFamily=Chrome&checkIn=2023-07-28&checkOut=2023-07-29&rooms=1&adults=1&children=0&priceCur=PHP&los=1&textToSearch=Bonifacio%20Global%20City%20(BGC)%20%2F%20Taguig&travellerType=0&familyMode=off&productType=-1'
driver = webdriver.Chrome(drive_path)

driver.get(url)

C:\Users\ryana\AppData\Local\Temp\ipykernel_14808\4008203257.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(drive_path)


In [68]:
def scrollDown():
    lastHeight = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(0.5)
        newHeight = driver.execute_script("return document.body.scrollHeight")
        if newHeight == lastHeight:
            break
        lastHeight = newHeight

def getHotelLinks():
    scrollDown()
    sleep(7)
    links = driver.find_elements(By.XPATH, "//a[@class = 'PropertyCard__Link']")
    sleep(5)
    
    for i in range(len(links)):
        links[i] = links[i].get_property(name = 'href')

    try:
        next_button = driver.find_element(By.XPATH, "//button[@id = 'paginationNext']")
    except(exceptions.NoSuchElementException):
        next_button = None

    return (next_button, links)


In [69]:
links = []

while True:
    sleep(5)
    args = getHotelLinks()
    links = links + args[1]    
    if args[0] == None:
        break
    else:
        args[0].click()

In [70]:
len(links)

114

In [71]:
def getHotelFeaturesDataFrame():
    hotelID = driver.find_element(By.XPATH, "//p[@class ='HeaderCerebrum__Name']").text
    features = driver.find_elements(By.XPATH,"//div[@data-element-name ='facility-highlights-item']")
    sleep(3)
    counter = 0
    
    for feature in features:
        features[counter] = features[counter].text
        counter = counter + 1

    sub_fac_df = pd.DataFrame(pd.get_dummies(features).sum(axis = 0), index = features).T
    features.insert(0, 'hotel_ID')
    
    sub_fac_df['hotel_ID'] = hotelID
    sub_fac_df = sub_fac_df[features]

    return sub_fac_df

def getCustomerReviewsDataFrame():
    usernames = []
    user_rating = []
    date = []
    
    hotelID = driver.find_element(By.XPATH, "//p[@class ='HeaderCerebrum__Name']").text
    user_info = driver.find_elements(By.XPATH, "//div[@data-info-type='reviewer-name']")
    ratings = driver.find_elements(By.XPATH, "//div[@class = 'Review-comment-leftScore']")
    reviewDate = driver.find_elements(By.XPATH, "//span[@class = 'Review-statusBar-date ']")

    sleep(6)
    for i in range(len(user_info)):
        usernames.append(unidecode(user_info[i].text.split()[0]).strip())
        user_rating.append(float(ratings[i].text))
        date_format = reviewDate[i].text.split(' ')
        date.append(date_format[1] +' ' + date_format[2][:-1] + ' '+ date_format[3])
        
    user_df = pd.DataFrame({
    'userID': usernames,
    'Rating':user_rating,
    'hotel_ID': hotelID,
    'review_date': date
    })
    return user_df

In [72]:
features_df = pd.DataFrame(columns = ['hotel_ID'])
users_df = pd.DataFrame(columns = ['userID', 'Rating','review_date'])

In [73]:
counter = 0

while counter < len(links):
    try:
        driver.get(links[counter])
        sleep(5)
        scrollDown()
        sleep(randint(3,7))
        features_df = pd.concat([features_df,getHotelFeaturesDataFrame()], ignore_index= True)
        users_df = pd.concat([users_df, getCustomerReviewsDataFrame()], ignore_index = True)
        
        counter = counter + 1
        
    except:
        print('Exception Occured')
        sleep(30)
        driver.refresh()

Exception Occured


In [74]:
users_df.to_csv('user_ratings.csv', index=False)
features_df.to_csv('hotel_features.csv', index=False)

# Start Here

In [75]:
users_df = pd.read_csv('user_ratings.csv')
users_df

,userID,Rating,review_date,hotel_ID
0,Sandip,10.0,January 31 2023,F1 Hotel Manila
1,Duane,10.0,October 21 2022,F1 Hotel Manila
2,Peter,10.0,August 21 2022,F1 Hotel Manila
3,Ma.,10.0,November 05 2022,F1 Hotel Manila
4,Lyra,10.0,January 09 2023,F1 Hotel Manila
...,...,...,...,...
1081,Ma.cristina,10.0,June 29 2023,Muggle Stay Guest House - Bonifacio Global Cit...
1082,Rochyr,7.0,June 28 2023,Muggle Stay Guest House - Bonifacio Global Cit...
1083,Yen,1.0,June 22 2023,Muggle Stay Guest House - Bonifacio Global Cit...
1084,Teresa,8.0,June 11 2023,Muggle Stay Guest House - Bonifacio Global Cit...


In [76]:
hotelFeatures_df = pd.read_csv('hotel_features.csv')
hotelFeatures_df

,hotel_ID,Airport transfer,Car park,Free Wi-Fi in all rooms!,Front desk [24-hour],Fitness center,Swimming pool [outdoor],Check-in/out [express],Luggage storage,Valet parking,...,Dishes and dining utensils,Body wash,Nightclub,"Kitchen basics (cookware, oils, condiments)",Video streaming services,Microwave,Tours,Bar,Poolside bar,Internet [LAN]
0,F1 Hotel Manila,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Grand Hyatt Manila,1.0,NaN,1.0,1.0,1.0,NaN,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,OYO 563 Mytown New York,NaN,NaN,1.0,1.0,1.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NicePad near Venice Mall,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Seda Bonifacio Global City,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,1BR Suite in Uptown BGC,NaN,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
110,Morgan Suite BGC shortwalk4 Venice Canal Mall ...,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,Cozy 2 Bedroom in Mckinley Hill near BGC,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
112,Lovely Loft Unit with Golf View,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
